<a href="https://colab.research.google.com/github/KomalT2003/DiabeticFootDetection/blob/main/UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


# Assuming you have your dataset stored as image paths and corresponding labels
image_paths = ["/content/gdrive/MyDrive/DatasetKem/DFC/20240327_134758.jpg","/content/gdrive/MyDrive/DatasetKem/EarlyStage/KK 54 DM 2YR(2).jpg"]  # List of paths to your foot images
labels = ["dfc","mild"]  # List of corresponding labels (e.g., 'mild', 'severe', 'normal')

# Define constants for image resizing
desired_height = 256
desired_width = 256

# Load and preprocess images
def preprocess_image(image_path):
    # Load image
    image = cv2.imread(image_path)
    # Resize image
    image = cv2.resize(image, (desired_width, desired_height))
    # Normalize image (assuming RGB)
    image = image.astype('float32') / 255.0
    return image

# Preprocess all images in the dataset
preprocessed_images = [preprocess_image(image_path) for image_path in image_paths]


label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)

integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

onehot_encoder = OneHotEncoder(sparse=False)
onehot_labels = onehot_encoder.fit_transform(integer_encoded)

# Now, preprocessed_images contains your preprocessed images,
# and onehot_labels contains corresponding one-hot encoded labels


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:


# Define U-Net model architecture
def unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Decoder
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = layers.concatenate([conv2, up1], axis=3)

    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(up1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)
    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    up2 = layers.concatenate([conv1, up2], axis=3)

    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(up2)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = layers.Conv2D(3, 1, activation='softmax')(conv5)  # 3 classes: mild, severe, normal

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Assuming you have data in X_train (input images) and y_train (segmentation masks/labels)
# Assuming your labels are one-hot encoded

# Initialize and compile the model
input_shape = (height, width, channels)  # Define your input shape
model = unet(input_shape)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)

# Once trained, you can use the model to predict on new images
# Assuming you have a new image called 'new_image'
predicted_mask = model.predict(np.expand_dims(new_image, axis=0))

# Post-process the predicted mask to get the final prediction
# For example, you can threshold the probabilities to get the predicted class
predicted_class = np.argmax(predicted_mask, axis=-1)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

from tensorflow.keras.layers import Activation

# Convert lists to numpy arrays
X = np.array(preprocessed_images)
y = np.array(onehot_labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the U-Net model architecture for binary classification
def unet_binary(input_shape):
    inputs = Input(input_shape)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Add more convolutional and pooling layers as needed

    up1 = Concatenate()([UpSampling2D(size=(2, 2))(conv2), conv1])
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(up1)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(conv3)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv3)  # Output with sigmoid activation for binary classification

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Initialize and compile the model
input_shape = X_train[0].shape
model = unet_binary(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=10)

# Once trained, you can use the model for prediction on new images
# For example:
# new_image = preprocess_image("path/to/new/image.jpg")
# predicted_mask = model.predict(np.expand_dims(new_image, axis=0))


Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    